In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# from sklearn.preprocessing import RobustScaler

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read CSVs

In [ ]:
train_csv = pd.read_csv('../input/older-dataset-for-dont-overfit-ii-challenge/train.csv')

y = train_csv['target'] 
x = train_csv.drop(["id" ,'target'],axis=1)


In [ ]:
x.head(2)


In [ ]:
x.info()

In [ ]:
y = pd.DataFrame(y)
y.info()

In [ ]:
x.var()

In [ ]:
test_csv  = pd.read_csv('../input/older-dataset-for-dont-overfit-ii-challenge/test.csv')
test = test_csv.drop(["id"],axis=1)
test.info()

In [ ]:
test.info()

In [ ]:
print(x.isnull().values.any() ,'\n',test.isnull().values.any())

In [ ]:
test.info()

In [ ]:
x.shape


In [ ]:
y.shape


In [ ]:
test.shape

In [ ]:
import matplotlib.pyplot as plt
y.hist()


In [ ]:
plt.bar(range(2), (x.shape[0], test.shape[0]))
plt.xticks(range(2), ('train','test'))
plt.ylabel('Number of data') 
plt.title('Can we avoid overfitting')
plt.show()

In [ ]:
print('Distributions of the first 28 columns')
plt.figure(figsize=(12,12))
for i, col in enumerate(list(x.columns)[2:30]):
    plt.subplot(7,4,i+1)
    plt.hist(x[col])
    plt.title(col)

# **Scaling data**

In [ ]:
# from sklearn.preprocessing import StandardScaler
# x_final = x
# ss = StandardScaler().fit(x_final)
# ss.transform(x_final)
# x_final = pd.DataFrame(x_final)
#x_final

In [ ]:
from sklearn.preprocessing import RobustScaler
rbst = RobustScaler()
x_final = rbst.fit_transform(x)
test_final = rbst.fit_transform(test)


x_final = pd.DataFrame(x_final)
test_final = pd.DataFrame(test_final)

I will use first use grid search with models to choose best hyper parameters 


In [ ]:
#Split data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_final , y , train_size=0.85, test_size=0.15, random_state = 42 ,stratify = y)

**Model :1 - logistic regression**

**First use Grissearch to get best parameters for Logistic regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


random_state = 0
logreg_clf = LogisticRegression(solver='liblinear',random_state = random_state)
param_grid = {'class_weight' : ['balanced', None], 
                'penalty' : ['l2','l1'],  
                'C' : [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(estimator = logreg_clf, param_grid = param_grid , scoring = 'accuracy', verbose = 1, n_jobs = -1, cv = 35 )

grid.fit(x_train,y_train)

print("Best Score:" + str(grid.best_score_))
print("Best Parameters: " + str(grid.best_params_))

In [ ]:
#Based on the grid search i came out with best hyperparameters

from sklearn.linear_model import LogisticRegression

logreg_clf = LogisticRegression(solver='liblinear' , C =0.2 , class_weight='balanced' , 
                                penalty='l1' , random_state = 45)

logreg_clf.fit(x_train, y_train) 
y_pred_lr = logreg_clf.predict(x_val)

from sklearn.metrics import accuracy_score

accuracy_lg = accuracy_score(y_val,  y_pred_lr)

print('prediction accuracyy',accuracy_lg*100,'%')

print('model score' , logreg_clf.score(x_val, y_val)*100 ,'%')

**Model 2 : RandomForestClassifi**

**Grid search fot his model to get best hyperparameters**

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV

random_state = 42
RFC_model = RandomForestClassifier(random_state = random_state,n_estimators = 10 ,criterion = 'gini')

param_grid_RFC = {'max_depth' : [5,7,9,11], 
                'min_samples_leaf' : [1, 2,5,7,9] ,
                'class_weight' : ['balanced', 'balanced_subsample' ]}

grid_rf = GridSearchCV(estimator = RFC_model, param_grid = param_grid_RFC , scoring = 'roc_auc', verbose = 1, n_jobs = -1, cv = 35 )

grid_rf.fit(x_train,y_train)

print("Best Score:" + str(grid_rf.best_score_))
print("Best Parameters: " + str(grid_rf.best_params_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
RFC_model = RandomForestClassifier(random_state = 42 ,n_estimators = 100 ,criterion = 'gini' , 
                                   class_weight='balanced', max_depth = 9, min_samples_leaf= 7)
RFC_model.fit(x_train, y_train)

from sklearn.metrics import accuracy_score
y_pred_rf = RFC_model.predict(x_val)
accuracy_rf = accuracy_score(y_val,  y_pred_rf)
print('Validation accuracyy',accuracy_rf*100,'%')
print('model score' , RFC_model.score(x_train, y_train))


**Model 3 : KNeighborsClassifier**

**Try grid search first**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

random_state = 42
knn_model = KNeighborsClassifier(metric ='minkowski' ,metric_params = None ,  leaf_size=30)

param_grid_knn = {'n_neighbors' : [3,4,5,7],
                  'weights':['uniform', 'distance'] , 
                  'p' :[2,3,4,5] }

grid_knn = GridSearchCV(estimator = knn_model, param_grid = param_grid_knn , verbose = 1, n_jobs = -1, cv = 20 )

grid_knn.fit(x_train,y_train)

print("Best Score:" + str(grid_knn.best_score_))
print("Best Parameters: " + str(grid_knn.best_params_))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kn_model = KNeighborsClassifier(metric ='minkowski' ,metric_params = None ,  leaf_size=2 , 
                                n_neighbors= 3, p=2,weights='uniform')
kn_model.fit(x_train, y_train)

from sklearn.metrics import accuracy_score

y_pred_kn = kn_model.predict(x_val)

accuracy_kn = accuracy_score(y_val,  y_pred_kn)
print('Prediction accuracyy',accuracy_kn*100,'%')
print('Train score' , kn_model.score(x_train, y_train))

# **Submission**

In [ ]:

y_pred_lr_test = logreg_clf.predict(test_final) #to be saved
y_pred_lr_test = pd.DataFrame({"ID": test_csv["id"],"Target": y_pred_lr_test})


y_pred_lr_test.to_csv('submission.csv', index=False)  
y_pred_lr_test

In [ ]:
y_pred_lr_test['Target'].hist()

In [ ]:
#10